In [ ]:
import sys
import numpy as np
from numpy import random
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import os
import os.path as osp
import pandas as pd

# Custom modules
sys.path.append(osp.join(os.getcwd(),"src")) # Add src subdirectory to python path
from data_funcs import synthetic_data

TODO:
* Table of Contents and Define key terms
    * Residual sum of squares RSS
* Figure labeling
* Is it ok to use bullet points and notebooks

# Custom Loss Functions for Fuel Moisture Models

*Author:* Jonathon Hirschi

[Fuel moisture content](https://www.ncei.noaa.gov/access/monitoring/dyk/deadfuelmoisture) is a measure of the water content of burnable materials. A loss function is intended to measure the fitting accuracy of a statistical model.

The purpose of this notebook is to discuss training fuel moisture models with various loss functions to try to account for the nonlinear effect of fuel moisture on wildfire rate of spread.

## Example with Weighted Least Squares

To illustrate the effect of changing loss functions, consider a simple linear model with one predictor for $n$ samples:

$$f(x_i, \pmb\beta) = \beta_0 + \beta_1 x_i, \text{ for }i=1,..., n$$

Ordinary least squares (OLS) is the most basic method of estimating the $\beta$ parameter values. The method minimizes the residual sum of squares (RSS), the loss function in this case. Equal weight is given to each residual value in the loss function,

$$r_i = y_i - f(x_i, \pmb \beta)$$

$$\pmb{\hat\beta_{OLS}} = argmin_\beta \sum_{i=1}^n r_i^2$$

Weighted least squares minimizes the weighted residual sum of squares, with a weight $w_i$ applied to each residual value. In principle, the weights could come from anywhere. 

$$\pmb{\hat\beta_{W}} = argmin_\beta \sum_{i=1}^n w_ir_i^2$$

In the following example, the OLS model for simulated data is compared to two different weighting schemes. In both cases, the weights come from normal distributions centered at the observed mean of the response value, but one has a large variance and the other has a much smaller variance. In this formulation of weights, as the variance of the normal distribution increases, the resulting model parameters approach the OLS parameters. 

In [ ]:
# Simulate Data
random.seed(123)
npts = 200
x = np.linspace(0, 100, npts)

y = 100+x+random.normal(0, 10, npts)

In [ ]:
# Fit OLS model
xx = x.reshape(len(x), 1)
w1 = np.ones(len(x)) # adding weights of 1 for illustrative purposes
model1 = LinearRegression().fit(xx, y, w1)
preds1 = model1.predict(xx)

In [ ]:
# Set up weighting distributions
m = np.floor(np.mean(y)) # center weights at central tendency of response data 
s = np.std(y)
rv = norm(loc=m, scale = s/2) 
rv2 = norm(loc=m, scale = 1) # extreme weight distribution for illustrative purposes

In [ ]:
# Fit Weighted LS
w2 = rv.pdf(y)
w2 = w2 / np.sum(w2) # normalize weights to sum to 1, not strictly necessary in this example 
model2 = LinearRegression().fit(xx, y, w2)
preds2 = model2.predict(xx)

w3 = rv2.pdf(y)
w3 = w3 / np.sum(w3) # normalize weights to sum to 1, not strictly necessary in this example 
model3 = LinearRegression().fit(xx, y, w3)
preds3 = model3.predict(xx)

In [ ]:
# Plot regression lines
sns.set(style='whitegrid')
p=sns.lineplot(x=x, y=preds1, label="OLS")
p=p.set(xlabel="X", ylabel="f(X)")
sns.lineplot(x=x, y=preds2, label = "Weighted (large variance)")
sns.lineplot(x=x, y=preds3, label = "Weighted (small variance)")
sns.scatterplot(x=x, y=y, alpha=.7)
plt.legend()
plt.title("OLS vs Weighted")

In [ ]:
# Plot Weighting Distriubtions
yy = np.sort(y)
fig, (ax1, ax2) = plt.subplots(2, figsize=(6, 6))
fig.suptitle('Weight Distributions')
ax1.plot(rv.pdf(yy), yy, color=sns.color_palette()[1])
ax1.set_title("Large Variance")
ax1.set_xticklabels([]);
# ax1.set_yticklabels([]);
ax2.plot(rv2.pdf(yy), yy, color=sns.color_palette()[2])
ax2.set_title("Small Variance")
ax2.set_xticklabels([]);
# ax2.set_yticklabels([]);
# plt.show

The weighted least squares model with the small variance gives tiny weight to residuals far away from the sample mean of the response.

## Fuel Moisture Nonlinear Effect on Rate of Spread

[Rate of spread](https://www.nwcg.gov/course/ffm/fire-behavior/83-rate-of-spread#:~:text=The%20rate%20of%20spread%20is,origin%20quickly%20with%20great%20intensity.) (ROS) is a measure of the speed a fire moves (often units of m/s). The following image shows the nonlinear relationship between FM and ROS at a single spatial location, while holding other variables associated with ROS constant. Wildfire spreads most readily in dry fuels, as seen in the peak of the ROS curve at zero FM. The ROS drops off quickly as fuels get wetter, but then it levels off until the ROS is zero, or when the FM reaches the "extinction value". 

![](https://wiki.openwfm.org/mediawiki/images/9/9b/Grass-fmc.png)

If the goal of training fuel moisture models is to get more accurate forecasts of wildfire ROS, it is intuitive that models should be trained directly on ROS. Instead of using loss functions on FM, why not construct a loss function directly with ROS? First, wildfire ROS is a complicated multifaceted conce3pt, and it is conceptually much cleaner to directly model fuel moisture and combine it with other observations to get reliable ROS estimates. Mathematically, there are issues related to the following two facts:

1. FM is highly correlated in time.
2. ROS reaches extinction value relatively quickly for dead fuels.

Since fuel moisture content is the "percent of the dry weight of that specific fuel" ([from NOAA](https://www.ncei.noaa.gov/access/monitoring/dyk/deadfuelmoisture#:~:text=Fuel%20moisture%20is%20a%20measure,content%20would%20be%20zero%20percent.)), this value can easily go over 100% for very wet fuels, since the water content can weight more than the underlying burnable material. The extinction value for tall grass, depicted above, reaches its extinction moisture value at roughly 25%. Thus, the ROS for tall grass with 25% FM would be the same as that of tall grass with 150% FM. In both cases, the ROS would be zero. 

This fact, combined with the temporal correlation of FM, makes it potentially undesirable to train FM models directly on ROS. Consider a case when the true FM content was 30% for tall grass. Model 1 predicts 25% FM and Model 2 predicts 150%. Both models would receive a loss of zero for that prediction, since both models predict zero ROS which matches the observed value. However, if atmospheric conditions led to the fuel drying out over time, the models would predict very different ROS within a few hours. Fuels with an FM of 25% would dry out relatively quickly compared to fuels with an FM of 150%, and thus the ROS would be nonzero in the former case much quicker than the latter. 

### Simulated FM Example

Fuel moisture content has a cyclical pattern that corresponds to a 24 hour day, where there are cycles throughout the day of temperature and relative humidity (two of the main theoretical drivers of FM). For this example, we simulate a sinusoidal curve with a decreasing linear trend.

In [ ]:
# Sim data, no rain for simplicity
random.seed(456)

hours = 400 # Total number of time steps
dat = synthetic_data(max_rain = 0, data_noise = .5)  # Sim data from FMDA project code
fm = dat['fm'][0:hours]

# Manually edit sim data to illustrate point about ROS
fm = fm + 20 - .07*np.arange(0, hours) # Shift up by 20, add decreasing trend

In [ ]:
h=np.arange(0, hours)
plt.plot(h, fm, "-o", markersize=3)
plt.title("Simulated FM")
plt.xlabel("Time (hour)")
plt.ylabel("FM (%)")

We then construct an idealized rate of spread curve from the source above. *Note:* an extinction moisture of about 25 is common for various fuel types.

In [ ]:
# Construct Idealized ROS curve
x = np.array([0, 5, 10, 15, 20, 25])
y = np.array([.05, .025, .02, .018, .01, 0])
from scipy.interpolate import CubicSpline
xvals = np.linspace(start=0, stop=30, num=100)

ros_f = CubicSpline(x, y)
def ros(fm):
    r = ros_f(fm)
    r[fm>25]=0
    return r



plt.plot(xvals, ros(xvals), "red")
plt.xlabel("Fuel Moisture (%)")
plt.ylabel("Rate of Spread (m/s)")
plt.title("ROS Curve")

Using the previously plotted simulated FM data, we now plot the ROS transformation of this data. Notice that many FM observations near the start of the time period get mapped to zero.

In [ ]:
# Plot ROS over time
rs = ros(fm)
plt.plot(h, rs, "red")
plt.xlabel("Time (hour)")
plt.ylabel("ROS (m/s)")
plt.title("ROS over Time")

### Autoregression Model on Simulated Data

We then train a simple Autoregressive model. Autoregression (AR) relates the response variable at time $t$ to response values at previous time steps, or "lags". The mathematical form of an AR model with time lags $k=1, 2, ..., K$:

$$
y_t = \beta_0 + \sum_{k=1}^K \beta_k y_{t-k} +\epsilon_t
$$

The terms here are:
* $y_t$: Response value at time $t$
* $\beta_0$: Parameter representing the overall mean, or intercept in the linear model context.
* $\beta_k$: Regression coefficient for time lag $k$.
* $y_{t-k}$: Observed response value at time $t-k$.
* $\epsilon_t$: Random error component at time $t$. Generally, this is assumed to be normally distributed with mean zero and identically distributed for all times. Given a variance parameter $\sigma^2$, the noise term is therefore $\epsilon_t\sim N(0, \sigma^2), \text{ for }t=1,2, ...$

Next, we examine autocorrelation plots to get a reasonable value for the number of time lags to examine. Fuel moisture is highly correlated in time, and there is a cyclical effect of the 24 hour weather cycles in a day. The raw autocorrelation plot below (displayed first in order) shows strong correlation across all times. However, the partial autocorrelation plot after that shows the first few lags represent most of that correlation, though the cyclical effect is still evident. Note that there is a small significant peak at roughly 24 hours.

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(fm)

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(fm, lags=50);

In [ ]:
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error

In [ ]:
# Model 1 fit with OLS on FM
h = h.reshape(-1, 1)
h2 = 300
fmtr=fm[0:h2]
fmte=fm[h2:len(fm)]
lags = 48
ar1 = AutoReg(fmtr, lags=lags).fit() # training fuel moisture is response
fit1 = ar1.predict(start=0, end=h2-1, dynamic=False)
fit1 = fit1[lags:h2]  # ignore NAN values at beginning when lags can't be calculated
preds1 = ar1.predict(start=len(fmtr), end=len(fmtr)+len(fmte)-1, dynamic=False)

In [ ]:
plt.plot(h, fm, "-o", markersize=3)
plt.plot(h[lags:h2], fit1, label="Training")
plt.plot(h[h2:len(h)], preds1, label="Forecast")
plt.legend()
plt.title("Linear Fit on OLS FM Loss")
plt.xlabel("Hour")
plt.ylabel("FM")
# plt.annotate("Training R2="+str(np.round(fmmod1.score(Xtr, fmtr), 3)), [325, 30])
# plt.annotate("Testing R2="+str(np.round(fmmod1.score(Xte, fmte), 3)), [325, 25])

Next, transform the fitted and forecasted values from the previous model to rate of spread. 

In [ ]:
rs=ros(fm)
rstr=rs[0:h2]
rste=rs[h2:len(rs)]

In [ ]:
plt.plot(h, rs)
plt.plot(h[lags:h2], ros(fit1), label="Training")
plt.plot(h[h2:len(h)], ros(preds1), label="Forecast")
plt.legend()
plt.title("Linear Fit on ROS FM Loss")
plt.xlabel("Hour")
plt.ylabel("FM")

A visual examination shows the model does a decent job of forecasting the ROS into the future.

Next, we fit an AR model of the same for directly on the transformed response variable. So here, ROS is the response.

In [ ]:
# Model 2 fit OLS on ROS
ar2 = AutoReg(rstr, lags=48).fit() # ROS is modeled response
fit2 = ar2.predict(start=0, end=h2-1, dynamic=False) # ignore NAN values at beginning when lags can't be calculated
fit2 = fit2[lags:h2]  # ignore NAN values at beginning when lags can't be calculated
preds2 = ar2.predict(start=len(rstr), end=len(rstr)+len(rste)-1, dynamic=False)

In [ ]:
plt.plot(h, rs)
plt.plot(h[lags:h2], fit2, label="Training")
plt.plot(h[h2:len(h)], preds2, label="Forecast")
plt.legend()
plt.title("Linear Fit on ROS Loss")
plt.xlabel("Hour")
plt.ylabel("ROS")
# plt.annotate("R2="+str(np.round(fmmod2.score(X, rs), 3)), [0, .025],
            # color=(0.8666666666666667, 0.5176470588235295, 0.3215686274509804))

Visually, the model performs worse in the forecasting period. Next, we examine the RMSE, with respect to ROS, for the training and forecasting periods for both models.

In [ ]:
df = pd.DataFrame({
    "Modeled Response": ["FM", "ROS"],
    "Training RMSE": [
        mean_squared_error(ros(fit1), rstr[lags:h2], squared=False), # RMSE with squared = False
        mean_squared_error(fit2, rstr[lags:h2], squared=False) 
    ],
    "Forecast RMSE": [
         mean_squared_error(ros(preds1), rste, squared=False), # RMSE with squared = False
        mean_squared_error(preds2, rste, squared=False) 
    ]
})
df

The AR model trained directly on ROS performs worse than the model trained on FM for both training errors and forecast errors. Also, note that the forecast error is over 4 times the training error for the model trained on ROS, while it is roughly 1.5 times for the FM model. The model trained on ROS is therefore overfitting more than the other model.

This is a simple example that shows it is not always desirable to train models directly on the quantity of interest. In this case, it is better to train directly on FM, then transform the predictions to ROS. 

The ROS transformation of FM effectively "throws out" information by collapsing a whole range of the outcome variable to zero (the extinction moisture). I hypothesize that it is better to train on FM than ROS for this reason. It is possible that simple AR models are ill equiped at modeling the ROS curve. So more models need to be investigated before we can confidently say that it is preferrable to train on FM. 

## Proposed Custom Loss Functions

In order to get the most accurate ROS forecasts, we have so far presented two approaches that can be thought of as opposite extremes. One approach is to train models with FM as the response and minimize the typical Residual Sum of Squares (RSS). After the initial training, we then transform the forecasts from FM to ROS. The drawback of this approach is that the training procedure will minimize errors across the entire observed range of FM values, and we might miss out on models that perform better for very dry fuels. The other approach is to transform FM observations to ROS first, and then train models to minimize the RSS of the ROS values. The drawback of this approach is that the ROS transformation collapses many FM observations to zero, at the extinction moisture level, so we lose information that is potentially useful for time dependent forecasts. I will refer to these two loss function approaches as the "baseline" loss functions. The goal of this project is to construct loss functions that get the best of both of baseline loss functions. In other words, we want to train FM models to be most accurate for very dry fuels without ignoring the information that saturated fuels give us.

### Weighted Average of Baseline Loss Functions

Let the size of the training sample be $n$. For each observed response value $y_1, y_2, ..., y_n$, the baseline methods assign a weight to each model residual. Then the model is trained to minimize the following weighted residual sum of squares. If the loss function is the RSS on FM values, the weights are uniform and the minimization procedure is equivalent to the unweighted case. Note that the weights will sum to 1 in this case.

$$w_i = \frac{1}{n} \text{ for all }i = 1, 2, ..., n$$
$$\sum_{i=1}^n w_i = 1$$


The other baseline case, where the FM observations are transformed to ROS, is equivalent to a weighted least squares procedure with the weights coming from the density given by the FM versus ROS curve. The weights are then normalized to sum to one. As a simple example, suppose we have observed FM values of $0, 25,$ and $30$. We visualize those points on the ROS curve below:

In [ ]:
y = np.array([0, 25, 30])
plt.plot(xvals, ros(xvals), "red")
plt.plot(y, ros(y), 'bo', label = "Observed FM values")
plt.xlabel("Fuel Moisture (%)")
plt.ylabel("Rate of Spread (m/s)")
plt.title("ROS Curve")
plt.legend()

This would lead to unnormalized weights of $0.05, 0, $ and $0$.

...

### Gamma Function 


## Proposed Experiment

My research hypothesis is that a weighted loss function for training fuel moisture models, which places more weight on lower values of FM, will result in more accurate ROS forecast than either an unweighted loss function on FM or a loss function directly on ROS. The weighting scheme is intended to assign a larger loss to model errors for low values of fuel moisture, since small errors in FM in that range lead to large differences in ROS. The fully realized version of this project would compare ROS forecasts to real observed fires with various fuel types. For a Masters' Project, I will consider the theoretical ROS at single spatial locations with only 10-hour dead fuel moisture (fm10). The fuel moisture models will be evaluated by their ROS forecast accuracy. Additionally, I need to specify spatial and temporal domains for analysis. To test my hypothesis in the simplified framework, I propose the following experimental design. 

1. Build a Testing Dataset:

**Spatial Frame**: Collect hourly fm10 observations for all RAWS stations in Colorado with complete data to run fuel moisture models (46 in total). 

**Time Frame**: Collect fm10 observations from May through September, corresponding to the Colorado Wildfire Season as defined by the [Western Fire Chiefs Association](https://wfca.com/articles/colorado-fire-season/). These time periods will be divided into 5 periods of training and testing, one for each month. The **training period** will be days 1-20 of each month, and the **testing period** will be days 21-30 or 31 of each month.

This will result in a dataset for 225 potential model runs (46 stations times 5 time periods).

2. Define a set of Fuel Moisture Models:

I will consider a few different models to examine how changing the loss function changes the ROS forecast accuracy:

* Simple Linear Regression Model: Including all FM variables as predictors. I include this because it will be simple to construct and deploy and analyze how it interacts with different loss functions.
* Auto-regressive Linear Model: The only predictor will be the 1 hour time lag of FM. I developed such a model for Data 2 Policy, which showed relatively strong accuracy. I include it to investigate how the loss functions interact with a time-series based model.
* Physics-Initiated Recurrent Neural Network: This model was developed by Jan Mandel and it is part of my ongoing research to assist him with developing this model.

3. Build a set of Candidate Loss Functions:

* Unweighted RSS on FM. This would be one loss function.
* Unweigthted RSS on ROS. This would also be one loss function.
* Weighted RSS on FM. This would be a set of loss functions. I will consider Gamma distributions with various parameter pairs. I select Gamma distributions a priori: they have positive support (FM cannot be negative) and they are flexible to get different shapes of the resulting distributions.

My hypothesis is that some loss function from the third set of loss functions will lead to the best forecasts of ROS.           

4. Fit the models and forecast ROS:

For each model and each loss function, train the model on 20 days of data, for the spatial locations I describe above, and forecast ROS 10 days into the future.

5. Evaluate Forecast Accuracy:

For each model run, calculate the accuracy of the ROS forecast by calculating the root mean squared error (RMSE) of the resulting ROS predictions. I will then summarize the average RMSE at each location and time period. Following that, I will analyze whether there were systematic differences between the different FM models, e.g. whether one loss function work better for the RNN than for the linear regression.


**Jan NOTES 10-1**
Train on 10 hour, but define ROS for 1 of 13 fuel categories. Discuss how to go to 1hr and 100hr.

## References

* Open Wildland Fire Modeling E Community. https://wiki.openwfm.org/wiki/
* National Wildfire Coordinating Group (NWCG). https://www.nwcg.gov/course/ffm/
* *Dead Fuel Moisture*, NOAA National Centers for Environmental Information. https://www.ncei.noaa.gov/access/monitoring/dyk/deadfuelmoisture